In [ ]:
pip install -r requirements.txt

# Import


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from plotly.subplots import make_subplots
import pickle
import plotly.graph_objects as go
import plotly.express as px

import charset_normalizer
import fuzzywuzzy
from fuzzywuzzy import process
import os
import re
from dateutil import parser

import warnings

warnings.filterwarnings("ignore")
plt.style.use(
    "https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle"
)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Character encoding


In [2]:
df_name = []
encoding = []
path = []

dataset_folder = "test"
for file in os.listdir(dataset_folder):
    file_path = os.path.join(dataset_folder, file)
    path.append(file_path)
    # look at the first ten thousand bytes to guess the character encoding
    with open(file_path, "rb") as rawdata:
        result = charset_normalizer.detect(rawdata.read())
        encoding.append(result["encoding"])
    # check what the character encoding might be
    file_name = os.path.splitext(file)[0]
    df_name.append(file_name)
    print(f"{file_name}: {result}")

context: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
mobile_plan_attr: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
mobile_plan_user: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
user: {'encoding': 'utf-8', 'language': 'Spanish', 'confidence': 1.0}


In [3]:
data_source = (tuple(df_name), tuple(encoding), tuple(path))

In [4]:
context = pd.read_csv(data_source[2][0], encoding=data_source[1][0])
context.sample(5)

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,direction
990,13186,Travel,%~Fri!e nd(s),!%Ra%iny,7 PM,1,1,0,0,1,0,0
271,1143,Travel,Friend(s?),S!u&nny,13:00,1,0,0,0,1,0,0
60,8231,Travel,Friend(s),Sunny,9:00,1,0,0,0,1,27,0
772,10014,Travel,!?A~lone,Sunny,11:00,1,1,0,0,1,8,0
860,8152,Travel,Frie&nd(s),S%!&nowy,20:00,1,1,0,0,1,0,0


In [5]:
mobile_plan_attr = pd.read_csv(
    data_source[2][1], encoding=data_source[1][1], delimiter=";"
)

mobile_plan_attr.sample(5)

,mobile_plan,description,price,duration
0,DATASILVER,"2GB/ day, high speed",100000,5d
2,SOCIALMEDIA,"1GB/ day\nUnlimited for Tik Tok, Facebook, You...",150000,3d
4,DATACALL,"2GB/ day, high speed\n300 mins call for extern...",200000,5d
3,SOCIALMEDIAGOLD,"3GB/ day, high speed\nUnlimited for Tik Tok, F...",250000,3d
1,DATAGOLD,"5GB/ day, high speed",200000,5d


In [6]:
mobile_plan_user = pd.read_csv(data_source[2][2], encoding=data_source[1][2])
mobile_plan_user.sample(5)

,id,coupon_id,accept
195,6528.0,DATASILVER,1.0
633,17015.0,SOCIALMEDIA,0.0
1386,16954.0,NaN,0.0
98,8449.0,SOCIALMEDIAGOLD,0.0
925,9611.0,DATACALL,0.0


In [7]:
user = pd.read_csv(data_source[2][3], encoding=data_source[1][3])
user.sample(5)

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,insta_freq,use_less_than_2GB,use_2GB_to_4GB
328,5130,이서영,Female,33,Masters,Computer & Mathematical,1615000000 VND,Married_1,KOREA,033-301-3748,발전 및 배전장치 조작원,1,4,6,2,0
550,13051,Irena Hande,Female,45,Masters,Management,76200$,Marri ed_1,DENMARK,04577343929,Koreanistiker,0,2,5,3,0
307,2175,박숙자,Female,57,Masters,Unemployed,282500000 VND,S ingle_0,KOREA,016-996-1319,단조기 조작원,0,10,1,1,1
942,3572,Joanna Wells,Female,62,Masters,Education&Training&Library,2497500000 VND,Sin gle_0,AUSTRALIA,0431 092 833,Biomedical scientist,1,2,0,1,0
698,15915,Sylvana Hiller,Female,62,Bachelor,Life Physical Social Science,1540000000 VND,M ar ried_1,DENMARK,+49(0)5559 97596,Krankenschwester,1,0,7,0,0


In [8]:
dataset = [context, mobile_plan_attr, mobile_plan_user, user]

# Cleaning

## Missing values check


In [9]:
for x in dataset:
  print('-----------------')
  print(x.isna().sum())

-----------------
id              0
purpose         0
go_with         0
weather         0
time            0
viettel_no_0    0
viettel_no_1    0
viettel_no_2    0
to_hanoi        0
to_other        0
score           0
direction       0
dtype: int64
-----------------
mobile_plan    0
description    0
price          0
duration       0
dtype: int64
-----------------
id            0
coupon_id    87
accept       92
dtype: int64
-----------------
id                     0
name                   0
gender                 0
age                    0
education            348
profession             0
income                 0
living_with            0
nation                 0
phone                  0
job                    0
fb_freq                0
yt_freq                0
insta_freq             0
use_less_than_2GB      0
use_2GB_to_4GB         0
dtype: int64


In [10]:
mobile_plan_user = mobile_plan_user.dropna(subset=['coupon_id', 'accept'])

## Duplicate


In [11]:
user['id'].duplicated().sum()

0

In [12]:
context['id'].duplicated().sum()

0

In [13]:
mobile_plan_user['id'].duplicated().sum()

130

In [31]:
mobile_plan_user[mobile_plan_user['id'].duplicated()].sort_values(by='id')

,id,coupon_id,accept
1155,1064.0,DATASILVER,0.0
1121,1331.0,SOCIALMEDIA,1.0
1120,1331.0,SOCIALMEDIA,1.0
1310,1625.0,SOCIALMEDIA,0.0
1332,1733.0,DATASILVER,0.0
...,...,...,...
1244,18459.0,DATACALL,1.0
1140,18936.0,SOCIALMEDIA,1.0
1275,19317.0,SOCIALMEDIAGOLD,0.0
1135,19696.0,SOCIALMEDIAGOLD,1.0


## Context


In [16]:
context.sample(5)

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,direction
660,13163,Travel,F!rie?n%d(s),%!Su?nny,11:00,1,1,0,0,1,0,0
573,15662,Travel,& Fam ily,Sunn~y,17:00,1,1,0,0,1,0,0
667,18431,Work,%Alone,Sunny,6:00,1,1,1,0,1,0,0
271,1143,Travel,Friend(s?),S!u&nny,13:00,1,0,0,0,1,0,0
899,2268,Work,!Al one,!&!R&ainy,7:00,1,1,1,0,1,34,0


In [17]:
for x in ["go_with", "weather"]:
    # Eliminate special characters and white spaces
    context[x] = context[x].apply(lambda x: re.sub(r"[^a-zA-Z\s]", "", x))

    # Lowercase all words
    context[x] = context[x].str.lower()

    print(context[x].unique())

['alone' 'friends' 'family' ' family' ' f amily' 'fami ly' 'frie nds'
 ' alone' 'al one' 'fam ily' 'a lone' 'f riends' ' friends' 'frien ds'
 'fa mily' 'friend s' 'alo ne' '  family' 'f amily' 'fr iends' 'fri e nds'
 'famil y' ' a lone' ' f riends' 'alon e' 'fri ends' '  alone'
 'frie   nds' 'a  lone' 'f r iends' 'al  one' 'al   one' 'frie n ds'
 'a   lone' 'f a mily' ' al one' 'fam  ily' 'fa  mily' ' friend s'
 ' fam ily' 'frien d s' 'friends ' 'f  riends' 'f r ien ds' ' fa mily'
 'fr ie nds' '  a lone' ' alo ne' 'friend  s' 'fr i ends' 'f rien ds'
 ' fami ly']
['sunny' 'rainy' 'snowy']


In [18]:
# List of possible values for the column 'go_with'
choices = ["alone", "friend(s)", "family"]


# Apply the fuzzy matching to the column 'go_with'
def correct_name(name):
    return process.extractOne(name, choices)[0]


context["go_with"] = context["go_with"].apply(correct_name)

In [19]:
# Eliminate white spaces
context["time"] = context["time"].str.replace(r"\s+", "", regex=True)


# Function to convert time to 24-hour format
def convert_to_24hr_format(time_str):
    # Use dateutil.parser to parse the time string
    return parser.parse(time_str).strftime("%H:%M")


# Apply the function to the column 'time'
context["time"] = context["time"].apply(convert_to_24hr_format)
context["time"] = pd.to_datetime(context["time"], format="%H:%M")

In [20]:
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

context['hour'] = pd.to_datetime(context['time']).dt.hour
context['time_of_day'] = context['hour'].apply(time_of_day)

context = context.drop(columns=['time', 'hour'])

In [21]:
context["purpose"] = context["purpose"].str.strip()
context["purpose"] = context["purpose"].str.lower()

In [22]:
context.purpose.unique()

array(['travel', 'visit', 'work'], dtype=object)

In [23]:
context["purpose"] = context["purpose"].replace('travel', 'visit')

In [24]:
context = context.drop(axis=1, columns=["to_hanoi", "to_other"])

In [25]:
context.sample(5)

,id,purpose,go_with,weather,viettel_no_0,viettel_no_1,viettel_no_2,score,direction,time_of_day
224,12404,work,alone,sunny,1,0,0,0,1,morning
664,1908,visit,friend(s),sunny,1,1,0,33,0,morning
817,14040,work,alone,sunny,1,0,0,0,1,morning
554,12537,visit,friend(s),snowy,1,0,0,28,0,afternoon
60,8231,visit,friend(s),sunny,1,0,0,27,0,morning


## User


In [36]:
user.sample(5)

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,insta_freq,use_less_than_2GB,use_2GB_to_4GB
978,14810,Sarah Hahn-Ullrich,Female,75,Bachelor,Education&Training&Library,290700$,Married_1,DENMARK,04218 04104,Programmierer,0,0,7,1,2
431,2918,Isadora Cavalcanti,Female,62,Associate,Office & Administrative Support,86400$,M ar ried_2,BRAZIL,(011) 9105-8200,Mágico,3,6,17,5,6
188,13706,Cecilia Ribeiro,Female,49,NaN,Arts Design Entertainment Sports & Media,872500000 VND,Married_0,BRAZIL,41 2584 1306,Profissional de marketing,1,8,1,2,0
161,11016,Jennifer Ramirez,Female,35,Highschool,Business & Financial,825000000 VND,Divorc ed_1,AUSTRALIA,0470-908-597,"Lighting technician, broadcasting/film/video",3,1,1,1,6
624,9932,김명자,Female,34,NaN,Computer & Mathematical,105300$,Married_0,KOREA,042-165-0492,약사 및 한약사,0,0,17,5,1


In [37]:
# Eliminate white spaces
user["living_with"] = user["living_with"].str.strip()
user["living_with"] = user["living_with"].str.replace(r"\s+", "", regex=True)
user["living_with"] = user["living_with"].str.lower()

In [38]:
user["nation"] = user["nation"].str.strip()
user["nation"] = user["nation"].str.lower()

In [39]:
user["gender"] = user["gender"].str.strip()
user["gender"] = user["gender"].str.lower()

In [40]:
user["education"] = user["education"].str.strip()
user["education"] = user["education"].str.lower()

In [41]:
user.nation.unique()

array(['korea', 'australia', 'brazil', 'denmark'], dtype=object)

In [42]:
user["nation"] = user["nation"].replace("australia", "oceania")
user["nation"] = user["nation"].replace(["brazil", "us"], "americas")
user["nation"] = user["nation"].replace(["denmark", "england", "russia"], "europe")
user["nation"] = user["nation"].replace(["korea", "japan", "china"], "asia")

In [43]:
user = user.rename(columns={"nation": "continent"})

In [44]:
currency_exchange_rate = 23000

# Function to clean and adjust income values
def clean_income(value):
    if 'VND' in value:
        return int(value.replace('VND', '')) / currency_exchange_rate
    elif '$' in value:
        return int(value.replace('$', ''))
    else:
        return int(value)

# Apply the function to the 'income' column
user['income_1'] = user['income'].apply(clean_income)

In [45]:
user['profession'] = user['profession'].str.lower()

In [46]:
user['profession'] = user['profession'].apply(lambda x: "jobs" if x not in ['student', 'retired', 'unemployed'] else x)

In [47]:
user["education"] = user["education"].fillna("unknown")
user["education"] = user["education"].replace("bachelor", "grad")
user["education"] = user["education"].replace(["associate", "highschool"], "undergrad")
user["education"] = user["education"].replace("masters", "postgrad")

In [48]:
user.living_with.unique()

array(['married_1', 'married_3', 'married_4', 'married_2', 'divorced_2',
       'divorced_1', 'single_0', 'single_3', 'single_1', 'married_0',
       'unmarried_0', 'unmarried_1', 'divorced_3', 'single_4',
       'widowed_2', 'widowed_3', 'unmarried_2', 'single_2', 'divorced_4',
       'unmarried_4', 'unmarried_3', 'divorced_0', 'widowed_1',
       'widowed_4'], dtype=object)

In [49]:
# Seperate Living_With column into Maritual_Status column and Children column
user[["marital_status", "children"]] = user["living_with"].str.split("_", expand=True)


user["marital_status"] = user["marital_status"].apply(
    lambda x: "relationship" if x == "married" else "single"
)

In [50]:
user['income_level'] = user['income_1'].apply(lambda x: 'lower' if x <= 30000 else 'lower-middle' if x > 30000 and x <= 58000 else 'middle' if x > 58000 and x <= 94000 else 'upper-middle' if x > 94000 and x <= 153000 else 'upper')

In [ ]:
user

## mobile_plan_attr


In [51]:
mobile_plan_attr.head()

,mobile_plan,description,price,duration
0,DATASILVER,"2GB/ day, high speed",100000,5d
1,DATAGOLD,"5GB/ day, high speed",200000,5d
2,SOCIALMEDIA,"1GB/ day\nUnlimited for Tik Tok, Facebook, You...",150000,3d
3,SOCIALMEDIAGOLD,"3GB/ day, high speed\nUnlimited for Tik Tok, F...",250000,3d
4,DATACALL,"2GB/ day, high speed\n300 mins call for extern...",200000,5d


In [52]:
mobile_plan_attr['mobile_plan'] = mobile_plan_attr['mobile_plan'].str.lower()

In [53]:
mobile_plan_attr['duration'] = mobile_plan_attr['duration'].str.replace('d', '')

In [54]:
mobile_plan_attr['capacity'] = mobile_plan_attr['description'].str.extract(r'(\d+)GB')

In [55]:
mobile_plan_attr.head()

,mobile_plan,description,price,duration,capacity
0,datasilver,"2GB/ day, high speed",100000,5,2
1,datagold,"5GB/ day, high speed",200000,5,5
2,socialmedia,"1GB/ day\nUnlimited for Tik Tok, Facebook, You...",150000,3,1
3,socialmediagold,"3GB/ day, high speed\nUnlimited for Tik Tok, F...",250000,3,3
4,datacall,"2GB/ day, high speed\n300 mins call for extern...",200000,5,2


## mobile_plan_user


In [56]:
mobile_plan_user.head()

,id,coupon_id,accept
0,7067.0,DATASILVER,1.0
1,2114.0,SOCIALMEDIAGOLD,1.0
2,6582.0,DATASILVER,1.0
3,11379.0,DATASILVER,0.0
4,3986.0,DATACALL,1.0


In [57]:
mobile_plan_user = mobile_plan_user.rename(columns={'coupon_id': 'mobile_plan'})

In [58]:
mobile_plan_user['mobile_plan'] = mobile_plan_user['mobile_plan'].str.lower()

In [59]:
mobile_planes = mobile_plan_user.mobile_plan.unique()

### Mobile plan recommendation


In [60]:
mobile_plan_rec_raw = mobile_plan_user[["id"]]

# Create columns for each item in the mobile_plan_user dataframe
for item in mobile_planes:
    mobile_plan_rec_raw[item] = mobile_plan_user["mobile_plan"].apply(
        lambda x: 1 if x == item else np.nan
    )

# Check rows that have duplicated ID
duplicate_rows = mobile_plan_rec_raw[
    mobile_plan_rec_raw.duplicated(subset=["id"], keep=False)
]
# Sort by ID
duplicate_rows.sort_values("id")

# Fill missing values of each ID group
# In each ID group, each missing value will be replaced by the last valid value forward (ffill), otherwise, by the next valid value backward (bfill).
mobile_plan_rec = duplicate_rows.groupby("id").apply(
    lambda x: x.fillna(method="ffill").fillna(method="bfill")
)
mobile_plan_rec = mobile_plan_rec.drop_duplicates()

mobile_plan_rec = mobile_plan_rec.drop(axis=1, columns="id").reset_index()
mobile_plan_rec = mobile_plan_rec.drop(axis=1, columns="level_1")

mobile_plan_rec = mobile_plan_rec.fillna(0)
mobile_plan_rec

,id,datasilver,socialmediagold,datacall,socialmedia,datagold
0,1064.0,1.0,0.0,0.0,0.0,0.0
1,1331.0,0.0,0.0,0.0,1.0,0.0
2,1625.0,0.0,0.0,0.0,1.0,0.0
3,1733.0,1.0,0.0,0.0,0.0,0.0
4,1756.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
113,18459.0,0.0,0.0,1.0,0.0,0.0
114,18936.0,0.0,0.0,0.0,1.0,0.0
115,19317.0,0.0,1.0,0.0,0.0,0.0
116,19696.0,0.0,1.0,0.0,0.0,0.0


### Mobile plan conversion rate


In [ ]:
mobile_plan_accept = mobile_plan_user.pivot_table(
index="id", columns="mobile_plan", values="accept", aggfunc="sum", fill_value=0
).reset_index()

# Đảm bảo tất cả các cột trong mobile_planes có mặt trong bảng kết quả cuối cùng
for plan in mobile_planes:
  if plan not in mobile_plan_accept.columns:
    mobile_plan_accept[plan] = 0

# Sắp xếp lại các cột theo thứ tự trong mobile_planes
mobile_plan_accept = mobile_plan_accept[["id"] + list(mobile_planes)]

In [ ]:
# Sử dụng pivot_table để tổng hợp dữ liệu
num_rec = mobile_plan_user.pivot_table(
    index="id", columns="mobile_plan", values="mobile_plan", aggfunc="count", fill_value=0
).reset_index()

# Đảm bảo tất cả các cột trong mobile_planes có mặt trong bảng kết quả cuối cùng
for plan in mobile_planes:
    if plan not in num_rec.columns:
        num_rec[plan] = 0

# Sắp xếp lại các cột theo thứ tự trong mobile_planes
num_rec = num_rec[["id"] + list(mobile_planes)]

# Đảm bảo rằng cả hai bảng đều có cùng thứ tự cột
assert list(mobile_plan_accept.columns) == list(num_rec.columns)

# Tạo bảng mobile_plan_cr bằng cách chia mobile_plan_accept cho num_rec
mobile_plan_cr = mobile_plan_accept.copy()

# Tính tỷ lệ chuyển đổi (conversion rate)
for plan in mobile_planes:
    mobile_plan_cr[plan] = mobile_plan_accept[plan] / num_rec[plan]

mobile_plan_cr.fillna(0, inplace=True)

# Hiển thị bảng mobile_plan_cr
mobile_plan_cr

# Data type check


In [ ]:
dataset = [context, mobile_plan_attr, mobile_plan_cr, mobile_plan_rec, user]
for x in dataset:
    print("-------------------------")
    print(x.dtypes)

In [ ]:
mobile_plan_attr.duration = mobile_plan_attr.duration.astype(int)
mobile_plan_attr.capacity = mobile_plan_attr.capacity.astype(int)


mobile_plan_cr.id = mobile_plan_cr.id.astype(int)
mobile_plan_cr.datasilver = mobile_plan_cr.datasilver.astype(float)
mobile_plan_cr.socialmediagold = mobile_plan_cr.socialmediagold.astype(float)
mobile_plan_cr.socialmedia = mobile_plan_cr.socialmedia.astype(float)
mobile_plan_cr.datacall = mobile_plan_cr.datacall.astype(float)
mobile_plan_cr.datagold = mobile_plan_cr.datagold.astype(float)


mobile_plan_rec.id = mobile_plan_rec.id.astype(int)
mobile_plan_rec.datasilver = mobile_plan_rec.datasilver.astype(int)
mobile_plan_rec.socialmediagold = mobile_plan_rec.socialmediagold.astype(int)
mobile_plan_rec.socialmedia = mobile_plan_rec.socialmedia.astype(int)
mobile_plan_rec.datacall = mobile_plan_rec.datacall.astype(int)
mobile_plan_rec.datagold = mobile_plan_rec.datagold.astype(int)


user.children = user.children.astype(int)
user.income_1 = user.income_1.astype(int)

## Merge


In [ ]:
df = (
  user.merge(context, on="id", how="left")
  .merge(mobile_plan_rec, on="id", how="left", suffixes=("", "_rec"))
  .merge(mobile_plan_cr, on="id", how="left", suffixes=("", "_cr"))
)

df.isna().sum()

In [ ]:
df = df.dropna()

## Outliers


In [ ]:
to_drop = ['id', 'name', 'income', 'living_with', 'phone', 'job', 'income_1']

In [ ]:
df = df.drop(axis=1, columns=to_drop)

In [ ]:
df.columns

In [ ]:
categorical_columns = ['gender', 'education', 'profession', 'income_level', 'continent', 'marital_status', 'purpose', 'go_with', 'weather', 'viettel_no_0', 'viettel_no_1', 'viettel_no_2', 'direction', 'datasilver', 'datagold', 'socialmedia', 'socialmediagold', 'datacall', 'datasilver_cr', 'socialmediagold_cr', 'socialmedia_cr', 'datacall_cr', 'datagold_cr']

continuous_columns = df.drop(axis=1, columns=categorical_columns).columns

# Create a canvas with 4 columns and 4 rows
fig, axes = plt.subplots(3, 3, figsize=(8, 8))

# Draw boxplot on the canvas
for i, ax in enumerate(axes.flatten()):
    if i < len(continuous_columns):
        sns.boxplot(y=continuous_columns[i], data=df, ax=ax)
        ax.set_title(f'Boxplot of {continuous_columns[i]}')
        ax.set_ylabel(continuous_columns[i])
    else:
        ax.axis('off')

# Fit layout
plt.tight_layout()

plt.show()

In [ ]:
# for x in continuous_columns:
#   range = 3
#   Q1 = df[x].quantile(0.25)
#   Q3 = df[x].quantile(0.75)
#   IQR = Q3 - Q1
#   lower_bound = Q1 - range * IQR
#   upper_bound = Q3 + range * IQR

#   df = df[(df[x] >= lower_bound) & (df[x] <= upper_bound)]

#   plt.show()

---


# input


In [ ]:
labels = ['datasilver', 'datagold', 'socialmedia', 'socialmediagold', 'datacall']

# EDA

## Univariate


In [ ]:
class EDA:
    
    def row(self,data):
        # Display the number of rows and columns in the dataframe
        fig = make_subplots(rows=1, cols=2)
        fig.add_trace(go.Indicator(mode = "number", value = data.shape[0], number={'font':{'color': '#E58F65','size':100}}, title = {"text": "🧾 Rows<br><span style='font-size:0.8em;color:gray'>In the Dataframe</span>"}, domain = {'x': [0, 0.5], 'y': [0.6, 1]}))
        fig.add_trace(go.Indicator(mode = "number", value = data.shape[1], number={'font':{'color': '#E58F65','size':100}}, title = {"text": "⭕ Columns<br><span style='font-size:0.8em;color:gray'>In the Dataframe</span>"}, domain = {'x': [0.5, 1], 'y': [0, 0.4]}))
        fig.show()
    
    def border_msg(self,msg, indent=1, width=None, title=None):
        """Print message-box with optional title."""
        lines = msg.split('\n')
        space = " " * indent
        if not width:
            width = max(map(len, lines))
        box = f'╔{"═" * (width + indent * 2)}╗\n'  
        if title:
            box += f'║{space}{title:<{width}}{space}║\n'  
            box += f'║{space}{"-" * len(title):<{width}}{space}║\n'  
        box += ''.join([f'║{space}{line:<{width}}{space}║\n' for line in lines])
        box += f'╚{"═" * (width + indent * 2)}╝' 
        print('\033[92m'+'\033[1m')
        print(box)
        
    def distribution(self,x,title):
        # Plot the distribution of a numerical column
        plt.figure(figsize=(10,8))
        ax = sns.distplot(x, kde=False,bins=30)
        values = np.array([rec.get_height() for rec in ax.patches])
        norm = plt.Normalize(values.min(), values.max())
        colors = plt.cm.jet(norm(values))
        for rec, col in zip(ax.patches,colors):
            rec.set_color(col)
        plt.title(title, size=20, color='black')
        
    def run(self,df):
        
        self.row(df)
        if len(df)>0:
            
            object_df = df.select_dtypes('object').columns.tolist()
            int_df = df.select_dtypes('int').columns.tolist()
            bool_df = df.select_dtypes('bool').columns.tolist()
            float_df = df.select_dtypes('float').columns.tolist()

            if len(object_df)>0:
                
                print( '\033[1m'+"OBJECT TYPE")
                for col in object_df:
                    # Display information about object type columns
                    self.border_msg(' '*25+ col.upper() + ' '*25)
                    self.border_msg('There are {} unique values in {} column'.format(df[col].nunique(),col.upper()))
                    plt.figure(figsize=(10,5))
                    sns.countplot(y = col, data = df,
                                  order = df[col].value_counts().index)
                    plt.show()
                    
            if len(int_df)>0:
                
                print('\033[1m'+"INT TYPE")
                for col in int_df:
                    # Display information about integer type columns
                    self.border_msg(' '*25+ col.upper() + ' '*25)
                    self.border_msg('Average value is : {}'.format(df[col].mean()))
                    self.border_msg('Minumum value is : {}'.format(df[col].min()))
                    self.border_msg('Maximum value is : {}'.format(df[col].max()))
                    self.distribution(df[col],title=col)
                    if df[col].mean()>df[col].std():
                        print(self.border_msg("Normal distributed Data Located below mean"))
                        
                    elif df[col].mean()<df[col].std():
                        print(self.border_msg("Normal distributed Data Located above mean"))
                    else:
                        self.border_msg("Mean Equals Std Dev - Distribution is normal")
                        
                    fig = make_subplots(rows=1, cols=2)
                    fig.add_trace(go.Indicator(mode = "number", value = df[col].mean(), number={'font':{'color': '#E58F65','size':100}}, title = {"text": "📌 Mean<br><span style='font-size:0.8em;color:gray'></span>"}, domain = {'x': [0, 0.5], 'y': [0.6, 1]}))
                    fig.add_trace(go.Indicator(mode = "number", value = df[col].std(), number={'font':{'color': '#E58F65','size':100}}, title = {"text": "🖇 Standart dev<br><span style='font-size:0.8em;color:gray'></span>"}, domain = {'x': [0.5, 1], 'y': [0, 0.4]}))
                    fig.show()
                    plt.show()
                 

            if len(bool_df)>0:
                
                print('\033[1m'+"BOOL TYPE")
                for col in bool_df:
                    # Display information about boolean type columns
                    self.border_msg(' '*25+ col.upper() + ' '*25)
                    plt.figure(figsize=(10,5))
                    sns.countplot(y = col, data = df,
                                  order = df[col].value_counts().index)
                    plt.show()
                    
            if len(float_df)>0:
                
                print('\033[1m'+"FLOAT TYPE")
                for col in float_df:
                    # Display information about float type columns
                    for col in int_df:
                        self.distribution(df[col],title=col)
                        if df[col].mean()>df[col].std():
                            print(self.border_msg("Normal distributed Data Located below mean"))
                        
                        elif df[col].mean()<df[col].std():
                            print(self.border_msg("Normal distributed Data Located above mean"))
                        else:
                            self.border_msg("Mean Equals Std Dev - Distribution is normal")

In [ ]:
# frame = EDA().run(df)

In [ ]:
to_check = ['age', 'fb_freq', 'yt_freq', 'insta_freq', 'use_less_than_2GB', 'use_2GB_to_4GB', 'viettel_no_0', 'viettel_no_1', 'viettel_no_2', 'score', 'direction', 'datasilver', 'socialmediagold', 'socialmedia', 'datacall', 'datagold', 'datasilver_cr', 'socialmediagold_cr','socialmedia_cr', 'datacall_cr', 'datagold_cr']

# model


In [ ]:
df.columns

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from sklearn.pipeline import Pipeline

In [ ]:
# Set random seed
seed = 42

# Initialize classifiers
models = {
    'Random Forest Classifier': RandomForestClassifier(random_state=seed),
    'Gradient Boosting Classifier': GradientBoostingClassifier(random_state=seed),
    'AdaBoost Classifier': AdaBoostClassifier(random_state=seed),
    'Logistic Regression': LogisticRegression(random_state=seed),
    'SVC': SVC(),
    'KNeighbors Classifier': KNeighborsClassifier(),
    'XGBoost Classifier': XGBClassifier(random_state=seed)
}

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=seed)

# Prepare to collect results
results = []

# Initialize LabelEncoder dictionary
label_dict = {
    'labels': ['datasilver', 'datagold', 'socialmedia', 'socialmediagold', 'datacall'],
    'cr': ['datasilver_cr', 'datagold_cr', 'socialmedia_cr', 'socialmediagold_cr', 'datacall_cr']
}

In [ ]:
# Iterate through each label in label_dict['labels']
for i, x in enumerate(label_dict['labels']):
    print(f"Training model for label {x}...")

    current_value = label_dict['cr'][i]
    other = [value for value in label_dict['cr'] if value != current_value]
    features = df.drop(axis=1, columns=label_dict['labels'] + other)

    X = features
    y = df[x]

    # Encode categorical columns
    check = (X.dtypes == 'object') | (X.dtypes == 'category')
    object_columns = list(check[check].index)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), X.columns.difference(object_columns)),
            ('cat', OneHotEncoder(), object_columns)
        ],
        remainder='passthrough'
    )

    # Initialize variables to store the best model and its metrics
    best_model_name = None
    best_accuracy = 0.0
    best_hamming_loss = 1.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1_score = 0.0

    for name, model in models.items():
        accuracy_cv_scores = []
        hamming_cv_losses = []
        precision_scores = []
        recall_scores = []
        f1_scores = []

        # Train the model using KFold
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            pipeline = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('classifier', model)
            ])

            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)

            # Metrics
            acc = accuracy_score(y_test, y_pred)
            hamming = hamming_loss(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='micro')
            recall = recall_score(y_test, y_pred, average='micro')
            f1 = f1_score(y_test, y_pred, average='micro')

            accuracy_cv_scores.append(acc)
            hamming_cv_losses.append(hamming)
            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

        # Calculate the mean of each metric
        mean_accuracy = np.mean(accuracy_cv_scores)
        mean_hamming_loss = np.mean(hamming_cv_losses)
        mean_precision = np.mean(precision_scores)
        mean_recall = np.mean(recall_scores)
        mean_f1_score = np.mean(f1_scores)

        # Save the best model and its metrics
        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_model_name = name
            best_hamming_loss = mean_hamming_loss
            best_precision = mean_precision
            best_recall = mean_recall
            best_f1_score = mean_f1_score

    # Print the best model and its metrics
    print(f"Best model for label {x}: {best_model_name}")
    print(f"Mean Accuracy: {best_accuracy}")
    print(f"Mean Hamming Loss: {best_hamming_loss}")
    print(f"Mean Precision: {best_precision}")
    print(f"Mean Recall: {best_recall}")
    print(f"Mean F1-score: {best_f1_score}")
    print("*" * 50)

    # Save the results to a list
    results.append({
        'Label': x,
        'Best Model': best_model_name,
        'Mean Accuracy': best_accuracy,
        'Mean Hamming Loss': best_hamming_loss,
        'Mean Precision': best_precision,
        'Mean Recall': best_recall,
        'Mean F1-score': best_f1_score
    })

# Create a DataFrame to store the results
results = pd.DataFrame(results)

In [ ]:
results

---
